In [98]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

import codecs
from pandas import *

In [119]:
# anime_data = codecs.open('anime.csv', 'r', encoding='utf-8', errors='ignore').readlines()
# users_data = codecs.open('users.csv', 'r', encoding='utf-8', errors='ignore').readlines()

anime_data = read_csv('anime.csv')
users_data = read_csv('users.csv')

In [120]:
anime_id_list = anime_data["MAL_ID"].tolist()
anime_score_list = anime_data["Score"].tolist()
anime_genres_list = anime_data["Genres"].tolist()
anime_popularity_list = anime_data["Popularity"].tolist()
anime_name_list = anime_data["Name"].tolist()

In [121]:
import math

genres = ["Action", "Adventure", "Avant Garde", "Award Winning", "Boys Love", "Comedy", "Drama", "Fantasy", "Girls Love",
           "Gourmet", "Horror", "Mystery", "Police", "Romance", "Sci-Fi", "Shounen", "Slice of Life", "Space", "Sports", 
          "Supernatural", "Suspense", "Ecchi", "Erotica", "Hentai", "School", "Historical", "Mecha", "Magic", "Seinen", 
         "Shoujo Ai", "Shoujo", "Parody", "Super Power", "Military", "Dementia", "Demons", "Martial Arts", "Harem", "Game", "Psychological",
         "Samurai", "Vampire", "Cars", "Thriller", "Josei", "Unknown", "Yaoi", "Yuri", "Music", "Shounen Ai", "Kids"]

genres = sorted(genres)

genres_indexes = {}
for i in range(len(genres)):
    genres_indexes[genres[i]] = i


In [122]:
anime_dictionary = {}

for i in range(len(anime_id_list)): 
    anime_id = anime_id_list[i]
    anime_score = anime_score_list[i]
    a_genres = anime_genres_list[i]
    a_popularity = anime_popularity_list[i]
    a_name = anime_name_list[i]
    
    anime_dictionary[anime_id] = [anime_score, a_genres, a_popularity, a_name]

In [123]:
anime_dictionary

{1: ['8.78',
  'Action, Adventure, Comedy, Drama, Sci-Fi, Space',
  39,
  'Cowboy Bebop'],
 5: ['8.39',
  'Action, Drama, Mystery, Sci-Fi, Space',
  518,
  'Cowboy Bebop: Tengoku no Tobira'],
 6: ['8.24',
  'Action, Sci-Fi, Adventure, Comedy, Drama, Shounen',
  201,
  'Trigun'],
 7: ['7.27',
  'Action, Mystery, Police, Supernatural, Drama, Magic',
  1467,
  'Witch Hunter Robin'],
 8: ['6.98',
  'Adventure, Fantasy, Shounen, Supernatural',
  4369,
  'Bouken Ou Beet'],
 15: ['7.95', 'Action, Sports, Comedy, Shounen', 1003, 'Eyeshield 21'],
 16: ['8.06',
  'Comedy, Drama, Josei, Romance, Slice of Life',
  687,
  'Hachimitsu to Clover'],
 17: ['7.59',
  'Slice of Life, Comedy, Sports, Shounen',
  3612,
  'Hungry Heart: Wild Striker'],
 18: ['8.15',
  'Action, Cars, Sports, Drama, Seinen',
  1233,
  'Initial D Fourth Stage'],
 19: ['8.76',
  'Drama, Horror, Mystery, Police, Psychological, Seinen, Thriller',
  169,
  'Monster'],
 20: ['7.91',
  'Action, Adventure, Comedy, Super Power, Martia

In [104]:
# -----------------------------------------------------------------------------------------------

In [105]:
user_id_list = users_data["user_id"].tolist()
user_anime_id_list = users_data["anime_id"].tolist()
user_rating_list = users_data["rating"].tolist()

In [106]:
x_train, y_train = [], []

i = 0
while i < 300000:
    user_id = user_id_list[i]
    
    # Dictionary to keep track of user's viewed genres
    user_genre_list = {}
    user_feature = [0] * (len(genres) + 1)
    rating_count = 0
    rating_sum = 0
    start_index = i
    while user_id_list[i] == user_id:
        # Movie that would be good to reccomend
        if(user_rating_list[i] >= 7):
            genres_string = anime_dictionary[user_anime_id_list[i]][1]
            genres_array = genres_string.split(', ')
            # Count all of the genres watched
            for genre in genres_array:
                if genre not in user_genre_list:
                    user_genre_list[genre] = 1
                else:
                    user_genre_list[genre] += 1
            rating_count += 1
            if anime_dictionary[user_anime_id_list[i]][0] != "Unknown": 
                rating_sum += float(anime_dictionary[user_anime_id_list[i]][0])
        # Increment
        i += 1
    user_genre_list = sorted(user_genre_list.items(), key=lambda item: item[1], reverse=True)
    
    for j in range(math.ceil(.25 * len(user_genre_list))):
        user_feature[genres_indexes[user_genre_list[j][0]]] = 1
        
    if rating_count == 0: 
        user_feature[-1] = 0
    else: 
        user_feature[-1] = rating_sum / rating_count
    
    # Go through all the animes for the user
    while start_index < i: 
        # If no rating is given, skip
        if user_rating_list[start_index] == 0:
            start_index += 1
        
        movie_instance = [0] * (len(genres) + 1)
        
        # Storing whether good recommendation or not
        if user_rating_list[start_index] >= 7:
            y_train.append(1)
        else: 
            y_train.append(-1)
            
        # Creating the features for the movie instance
        genres_string = anime_dictionary[user_anime_id_list[start_index]][1]
        genres_array = genres_string.split(', ')
        
        for genre in genres_array:
            movie_instance[genres_indexes[genre]] = 1
            
        if anime_dictionary[user_anime_id_list[start_index]][0] == "Unknown": 
            movie_instance[-1] = 0
        else: 
            movie_instance[-1] = float(anime_dictionary[user_anime_id_list[start_index]][0])
        
        final_feature_arr = []
        for a in user_feature: 
            final_feature_arr.append(a)
            
        for b in movie_instance: 
            final_feature_arr.append(b)
            
        x_train.append(final_feature_arr)
        
        start_index += 1

In [107]:
x_training, x_testing, y_training, y_testing = train_test_split(x_train, y_train, test_size = 0.2, random_state=0)

mlp = MLPClassifier()
mlp.fit(x_training, y_training)
mlp.score(x_testing, y_testing)

0.7082824494068388

In [164]:
# input_data = read_csv('input.csv')

inp_ids = [0, 0, 0, 0,
          1, 1, 1, 1, 1, 
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
           3,3,3]
inp_anime_ids = [20, 38000, 38777, 16498,
                9756, 31733, 350, 11061, 106,
                8074, 20, 38000, 38777, 45, 37349, 11771, 4224, 9213, 30276,
                38000, 136, 16498]
inp_score = [8, 9, 10, 3,
            10, 7, 8, 1, 10,
            8, 10, 10, 7, 7, 9, 8, 8, 8, 9, 10,10, 9]
inp_watched_episodes = [150, 24, 3, 0,
                       100, 20, 80, 10, 100,
                       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 24 , 100, 50]

In [156]:
def generatepredictions(uid):
    best_anime_id, best_prediction = 1, 0
    best_popularity, best_rating = 100000, 0

    watched_anime_ids = []
    
    i = 0
    while i < len(inp_ids): 
        if inp_ids[i] == uid:
            break
        i += 1
    
    ind = i
    while ind < len(inp_ids) and inp_ids[ind] == uid:
        watched_anime_ids.append(inp_anime_ids[ind])
        ind += 1
    
    # Dictionary to keep track of user's viewed genres
    u_genre_list = {}
    u_feature = [0] * (len(genres) + 1)
    rating_count = 0
    rating_sum = 0
    while i < len(inp_ids) and inp_ids[i] == uid:
            
        # Movie that would be good to reccomend
        if(inp_score[i] >= 7):
            genres_string = anime_dictionary[inp_anime_ids[i]][1]
            genres_array = genres_string.split(', ')
            
            # Count all of the genres watched
            for genre in genres_array:
                if genre not in u_genre_list:
                    u_genre_list[genre] = 1
                else:
                    u_genre_list[genre] += 1
            rating_count += 1
            if anime_dictionary[inp_anime_ids[i]][0] != "Unknown": 
                rating_sum += float(anime_dictionary[inp_anime_ids[i]][0])
        # Increment
        i += 1
    u_genre_list = sorted(u_genre_list.items(), key=lambda item: item[1], reverse=True)

    for j in range(math.ceil(.25 * len(u_genre_list))):
        u_feature[genres_indexes[u_genre_list[j][0]]] = 1

    if rating_count == 0: 
        u_feature[-1] = 0
    else: 
        u_feature[-1] = rating_sum / rating_count

    # Go through all the animes
    for n in range(len(anime_id_list)):
            
        movie_instance = [0] * (len(genres) + 1)

        # Creating the features for the movie instance
        genres_string = anime_dictionary[anime_id_list[n]][1]
        genres_array = genres_string.split(', ')

        for genre in genres_array:
            movie_instance[genres_indexes[genre]] = 1

        if anime_dictionary[anime_id_list[n]][0] == "Unknown": 
            movie_instance[-1] = 0
        else: 
            movie_instance[-1] = float(anime_dictionary[anime_id_list[n]][0])

        final_feature_arr = []
        for a in u_feature: 
            final_feature_arr.append(a)

        for b in movie_instance: 
            final_feature_arr.append(b)

        prediction = mlp.predict([final_feature_arr])[0]
        
        if prediction == 1: 
            if anime_dictionary[anime_id_list[n]][0] != "Unknown":
                if float(anime_dictionary[anime_id_list[n]][0]) > best_rating - 0.5 and anime_dictionary[anime_id_list[n]][2] < best_popularity and anime_id_list[n] not in watched_anime_ids:
                    best_prediction = prediction
                    best_anime_id = anime_id_list[n]
                    best_popularity = anime_dictionary[anime_id_list[n]][2]
            
    return best_anime_id

In [157]:
# User 0 liked: Naruto, Demon Slayer, Jutsu Kaisen
recommended_anime = generatepredictions(0)
print(anime_dictionary[recommended_anime][3] + ": " + anime_dictionary[recommended_anime][1])

Hunter x Hunter (2011): Action, Adventure, Fantasy, Shounen, Super Power


In [158]:
# Juans personal recommendation
juan_recommended_anime = generatepredictions(1)
print(anime_dictionary[juan_recommended_anime][3] + ": " + anime_dictionary[juan_recommended_anime][1])

Shingeki no Kyojin: Action, Military, Mystery, Super Power, Drama, Fantasy, Shounen


In [159]:
mike_recommended_anime = generatepredictions(2)
print(anime_dictionary[mike_recommended_anime][3] + ": " + anime_dictionary[mike_recommended_anime][1])

Death Note: Mystery, Police, Psychological, Supernatural, Thriller, Shounen


In [165]:
Dr_recommended_anime = generatepredictions(3)
print(anime_dictionary[Dr_recommended_anime][3] + ": " + anime_dictionary[Dr_recommended_anime][1])

Tokyo Ghoul: Action, Mystery, Horror, Psychological, Supernatural, Drama, Seinen
